# Hari 6: Pengolahan Data Kategorikal dan Numerikal (Encoding dan Scaling)

Notebook ini akan membahas berbagai teknik untuk mengolah data kategorikal dan numerikal, yang merupakan langkah penting dalam tahap pra-pemrosesan data sebelum membangun model machine learning.

## 1. Import Libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer

## 2. Membuat Dataset Contoh

Kita akan membuat dataset contoh sederhana yang berisi data kategorikal dan numerikal.

In [3]:
import pandas as pd
# Membuat dictionary untuk data
data = {
    'Gender': ['Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female', 'Male', 'Female'],
    'City': ['New York', 'London', 'Paris', 'Tokyo', 'New York', 'Paris', 'London', 'Tokyo', 'Paris', 'New York'],
    'Age': [25, 30, 35, 28, 40, 22, 45, 33, 29, 38],
    'Income': [50000, 60000, 75000, 55000, 90000, 45000, 120000, 65000, 58000, 80000],
    'Product_Purchased': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C']
}

# Membuat DataFrame
df = pd.DataFrame(data)

print("Dataset Awal:")
print(df)

Dataset Awal:
   Gender      City  Age  Income Product_Purchased
0    Male  New York   25   50000                 A
1  Female    London   30   60000                 B
2    Male     Paris   35   75000                 A
3  Female     Tokyo   28   55000                 C
4    Male  New York   40   90000                 B
5  Female     Paris   22   45000                 A
6    Male    London   45  120000                 C
7  Female     Tokyo   33   65000                 B
8    Male     Paris   29   58000                 A
9  Female  New York   38   80000                 C


## 3. Pengolahan Data Kategorikal

Data kategorikal perlu diubah menjadi format numerik agar dapat diproses oleh algoritma machine learning. Dua teknik umum adalah Label Encoding dan One-Hot Encoding.

### 3.1. Label Encoding

Label Encoding mengubah setiap kategori unik menjadi sebuah angka integer. Cocok untuk variabel kategorikal ordinal (yang memiliki urutan).

In [4]:
# Membuat salinan DataFrame untuk Label Encoding
df_label_encoded = df.copy()

# Inisialisasi LabelEncoder
label_encoder = LabelEncoder()

# Terapkan Label Encoding pada kolom 'Product_Purchased' (asumsi ordinal)
df_label_encoded['Product_Purchased_Encoded'] = label_encoder.fit_transform(df_label_encoded['Product_Purchased'])

print("Dataset setelah Label Encoding (Product_Purchased):")
print(df_label_encoded[['Product_Purchased', 'Product_Purchased_Encoded']])

Dataset setelah Label Encoding (Product_Purchased):
  Product_Purchased  Product_Purchased_Encoded
0                 A                          0
1                 B                          1
2                 A                          0
3                 C                          2
4                 B                          1
5                 A                          0
6                 C                          2
7                 B                          1
8                 A                          0
9                 C                          2


### 3.2. One-Hot Encoding

One-Hot Encoding membuat kolom biner baru untuk setiap kategori unik. Cocok untuk variabel kategorikal nominal (yang tidak memiliki urutan) untuk menghindari asumsi urutan yang salah.

In [5]:
# Membuat salinan DataFrame untuk One-Hot Encoding
df_one_hot_encoded = df.copy()

# Menggunakan pandas get_dummies untuk One-Hot Encoding yang lebih mudah
df_one_hot_encoded = pd.get_dummies(df_one_hot_encoded, columns=['Gender', 'City'], prefix=['Gender', 'City'])

print("Dataset setelah One-Hot Encoding (Gender & City):")
print(df_one_hot_encoded.head())

Dataset setelah One-Hot Encoding (Gender & City):
   Age  Income Product_Purchased  Gender_Female  Gender_Male  City_London  \
0   25   50000                 A          False         True        False   
1   30   60000                 B           True        False         True   
2   35   75000                 A          False         True        False   
3   28   55000                 C           True        False        False   
4   40   90000                 B          False         True        False   

   City_New York  City_Paris  City_Tokyo  
0           True       False       False  
1          False       False       False  
2          False        True       False  
3          False       False        True  
4           True       False       False  


### 3.3. One-Hot Encoding menggunakan ColumnTransformer (lebih fleksibel)

ColumnTransformer memungkinkan kita menerapkan transformasi berbeda pada kolom yang berbeda secara bersamaan.

In [6]:
# Membuat salinan DataFrame
df_col_transformer = df.copy()

# Definisikan kolom kategorikal yang akan di-encode
categorical_features = ['Gender', 'City']

# Membuat ColumnTransformer
# 'ohe' adalah nama transformer, OneHotEncoder() adalah estimatornya, dan categorical_features adalah kolom yang akan ditransformasi
# remainder='passthrough' berarti kolom lain yang tidak disebutkan akan dibiarkan apa adanya
preprocessor = ColumnTransformer(
    transformers=[
        ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features)],
    remainder='passthrough')

# Terapkan ColumnTransformer
df_transformed_data = preprocessor.fit_transform(df_col_transformer)

# Mendapatkan nama kolom setelah One-Hot Encoding
# OneHotEncoder menghasilkan nama fitur baru, kita perlu mengambilnya
# Untuk fitur yang di-encode: preprocessor.named_transformers_['ohe'].get_feature_names_out(categorical_features)
# Untuk fitur sisa: kolom numerik dan 'Product_Purchased'
encoded_feature_names = preprocessor.named_transformers_['ohe'].get_feature_names_out(categorical_features)
remainder_feature_names = [col for col in df_col_transformer.columns if col not in categorical_features]
all_feature_names = list(encoded_feature_names) + remainder_feature_names

# Konversi hasil transformasi kembali ke DataFrame
df_col_transformed_final = pd.DataFrame(df_transformed_data, columns=all_feature_names)

print("Dataset setelah One-Hot Encoding menggunakan ColumnTransformer:")
print(df_col_transformed_final.head())

Dataset setelah One-Hot Encoding menggunakan ColumnTransformer:
  Gender_Female Gender_Male City_London City_New York City_Paris City_Tokyo  \
0           0.0         1.0         0.0           1.0        0.0        0.0   
1           1.0         0.0         1.0           0.0        0.0        0.0   
2           0.0         1.0         0.0           0.0        1.0        0.0   
3           1.0         0.0         0.0           0.0        0.0        1.0   
4           0.0         1.0         0.0           1.0        0.0        0.0   

  Age Income Product_Purchased  
0  25  50000                 A  
1  30  60000                 B  
2  35  75000                 A  
3  28  55000                 C  
4  40  90000                 B  


## 4. Pengolahan Data Numerikal (Scaling)

Scaling data numerikal penting untuk algoritma yang sensitif terhadap skala fitur, seperti SVM, KNN, dan regresi linear. Dua teknik umum adalah Normalisasi (Min-Max Scaling) dan Standardisasi (Z-score Scaling).

### 4.1. Normalisasi (Min-Max Scaling)

Mengubah skala fitur ke rentang tertentu, biasanya [0, 1].
Formula: `X_scaled = (X - X_min) / (X_max - X_min)`

In [7]:
# Membuat salinan DataFrame untuk Min-Max Scaling
df_min_max_scaled = df.copy()
numerical_cols = ['Age', 'Income']

# Inisialisasi MinMaxScaler
min_max_scaler = MinMaxScaler()

# Terapkan Min-Max Scaling pada kolom numerik
df_min_max_scaled[numerical_cols] = min_max_scaler.fit_transform(df_min_max_scaled[numerical_cols])

print("Dataset setelah Min-Max Scaling (Age & Income):")
print(df_min_max_scaled[numerical_cols].head())

Dataset setelah Min-Max Scaling (Age & Income):
        Age    Income
0  0.130435  0.066667
1  0.347826  0.200000
2  0.565217  0.400000
3  0.260870  0.133333
4  0.782609  0.600000


### 4.2. Standardisasi (Z-score Scaling)

Mengubah skala fitur sehingga memiliki rata-rata 0 dan standar deviasi 1.
Formula: `X_scaled = (X - mean(X)) / std(X)`

In [8]:
# Membuat salinan DataFrame untuk Standard Scaling
df_standard_scaled = df.copy()

# Inisialisasi StandardScaler
standard_scaler = StandardScaler()

# Terapkan Standard Scaling pada kolom numerik
df_standard_scaled[numerical_cols] = standard_scaler.fit_transform(df_standard_scaled[numerical_cols])

print("Dataset setelah Standard Scaling (Age & Income):")
print(df_standard_scaled[numerical_cols].head())

Dataset setelah Standard Scaling (Age & Income):
        Age    Income
0 -1.112485 -0.928892
1 -0.370828 -0.459755
2  0.370828  0.243951
3 -0.667491 -0.694323
4  1.112485  0.947657


## 5. Menggabungkan Encoding dan Scaling menggunakan ColumnTransformer

Kita dapat menggunakan ColumnTransformer untuk menerapkan encoding pada fitur kategorikal dan scaling pada fitur numerikal secara bersamaan.

In [9]:
# Membuat salinan DataFrame
df_full_preprocess = df.copy()

# Definisikan kolom kategorikal dan numerikal
categorical_features_final = ['Gender', 'City', 'Product_Purchased'] # Product_Purchased juga di-OHE di sini
numerical_features_final = ['Age', 'Income']

# Membuat pipeline untuk transformasi
# Untuk fitur kategorikal: OneHotEncoder
# Untuk fitur numerikal: StandardScaler
preprocessor_final = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_features_final),
        ('num', StandardScaler(), numerical_features_final)
    ])

# Terapkan preprocessor
df_processed_data = preprocessor_final.fit_transform(df_full_preprocess)

# Mendapatkan nama kolom setelah transformasi
encoded_cat_feature_names = preprocessor_final.named_transformers_['cat'].get_feature_names_out(categorical_features_final)
# Nama fitur numerik tidak berubah setelah StandardScaler
all_processed_feature_names = list(encoded_cat_feature_names) + numerical_features_final

# Konversi hasil transformasi kembali ke DataFrame
df_fully_processed = pd.DataFrame(df_processed_data, columns=all_processed_feature_names)

print("Dataset setelah Encoding Kategorikal dan Scaling Numerikal menggunakan ColumnTransformer:")
print(df_fully_processed.head())

Dataset setelah Encoding Kategorikal dan Scaling Numerikal menggunakan ColumnTransformer:
   Gender_Female  Gender_Male  City_London  City_New York  City_Paris  \
0            0.0          1.0          0.0            1.0         0.0   
1            1.0          0.0          1.0            0.0         0.0   
2            0.0          1.0          0.0            0.0         1.0   
3            1.0          0.0          0.0            0.0         0.0   
4            0.0          1.0          0.0            1.0         0.0   

   City_Tokyo  Product_Purchased_A  Product_Purchased_B  Product_Purchased_C  \
0         0.0                  1.0                  0.0                  0.0   
1         0.0                  0.0                  1.0                  0.0   
2         0.0                  1.0                  0.0                  0.0   
3         1.0                  0.0                  0.0                  1.0   
4         0.0                  0.0                  1.0                

## 6. Kesimpulan

Pengolahan data kategorikal (Encoding) dan numerikal (Scaling) adalah langkah krusial dalam pra-pemrosesan data. 
- **Label Encoding**: Cocok untuk data kategorikal ordinal.
- **One-Hot Encoding**: Cocok untuk data kategorikal nominal, menghindari asumsi urutan yang salah. `pd.get_dummies` atau `ColumnTransformer` dengan `OneHotEncoder` dari scikit-learn dapat digunakan.
- **Min-Max Scaling (Normalisasi)**: Menskalakan data ke rentang [0, 1]. Berguna jika distribusi data tidak Gaussian atau jika algoritma sensitif terhadap nilai absolut.
- **Standard Scaling (Standardisasi)**: Menskalakan data sehingga memiliki mean 0 dan standar deviasi 1. Berguna jika data mengikuti distribusi Gaussian atau jika algoritma mengasumsikan data terdistribusi secara normal (misalnya, regresi linear, regresi logistik, LDA).

Pemilihan teknik yang tepat tergantung pada jenis data dan algoritma machine learning yang akan digunakan.